In [9]:
import os
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

from helpers.UNET import UNET
from helpers.load import LoadandAugment

# Base path for the dataset
path = "/home/tfuser/project/Satelite/data/dataset/"
train_path = os.path.join(path, "train")
val_path = os.path.join(path, "val")
test_path = os.path.join(path, "test")

# Initialize data loaders
train_data = LoadandAugment(train_path, "train", 8)
val_data = LoadandAugment(val_path, "val", 8)

# Initialize UNET model
unet = UNET(input_shape=(512, 512, 4))
unet.model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Setup the model checkpoint callback to save the best model
checkpoint_callback = ModelCheckpoint(
    'best_model.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min'
)

# Train the model
unet.model.fit(
    train_data.dataset, 
    validation_data=val_data.dataset, 
    epochs=40, 
    callbacks=[checkpoint_callback]
)


Epoch 1/40
23/23 [==============================] - ETA: 0s - loss: 0.4711 - accuracy: 0.7561
Epoch 1: val_loss improved from inf to 14.72005, saving model to best_model.h5
23/23 [==============================] - 15s 365ms/step - loss: 0.4711 - accuracy: 0.7561 - val_loss: 14.7200 - val_accuracy: 0.2886
Epoch 2/40
23/23 [==============================] - ETA: 0s - loss: 0.3870 - accuracy: 0.8105
Epoch 2: val_loss improved from 14.72005 to 1.10219, saving model to best_model.h5
23/23 [==============================] - 8s 342ms/step - loss: 0.3870 - accuracy: 0.8105 - val_loss: 1.1022 - val_accuracy: 0.2881
Epoch 3/40
23/23 [==============================] - ETA: 0s - loss: 0.3632 - accuracy: 0.8230
Epoch 3: val_loss did not improve from 1.10219
23/23 [==============================] - 7s 285ms/step - loss: 0.3632 - accuracy: 0.8230 - val_loss: 1.1360 - val_accuracy: 0.2884
Epoch 4/40
23/23 [==============================] - ETA: 0s - loss: 0.3486 - accuracy: 0.8305
Epoch 4: val_loss di

In [11]:

import matplotlib.pyplot as plt
def plot_predictions(images, masks, predictions, num=3):
    plt.figure(figsize=(15, 5*num))
    
    for i in range(num):
        plt.subplot(num, 3, i*3+1)
        plt.imshow(images[i])
        plt.title("Satellite Image")
        plt.axis('off')
        
        plt.subplot(num, 3, i*3+2)
        plt.imshow(masks[i], cmap='gray')
        plt.title("Actual Mask")
        plt.axis('off')

        plt.subplot(num, 3, i*3+3)
        plt.imshow(predictions[i], cmap='gray')
        plt.title("Predicted Mask")
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()
    
# Load the best model
unet.model.load_weights('best_model.h5')

# Evaluate the model
test_data = LoadandAugment(test_path, "test", 4)
loss, acc = unet.model.evaluate(test_data.dataset)


6/6 [==============================] - 2s 43ms/step - loss: 0.4601 - accuracy: 0.8034
Test Loss: 0.46007630228996277, Test Accuracy: 0.8033626675605774
